In [4]:
from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.postprocessing import *
import os 


In [ ]:
folder = ''
list_dir = os.listdir(folder)
liste_inference = []
liste_study_uid = []
for file_ in list_dir : 
    liste_inference.append(os.path.join(folder, file_))
    liste_study_uid.append(file_.split('/')[0])

In [ ]:
csv_path= ''

with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    dataset = []
    for row in reader :
        dataset.append(row)
        
del dataset[0] #enlever première ligne

In [ ]:
#with study uid, find path pet 

liste_pet = []
for uid in liste_study_uid : 
    for row in dataset : 
        if uid in row : 
            liste_pet.append(row[-2])

#rename path pet 
root = ''
for path in liste_pet : 
    p = path.split('/')[:]
    path = os.path.join(root, p)


In [ ]:
 mip_directory = ''
 filenames = []

 for inference, pet, uid in zip(liste_inference, liste_pet, liste_study_uid) : 
    sub = []
    inference_array = read_inference(inference)
    #binarize_array = binarize(inference_array)

    pet_array, size = read_img(liste_pet)

    sub.append(mip_projection(pet_array, 0, mip_directory, uid, 'pet', cmap = 'Greys', vmin= 0 , vmax = 6))
    sub.append(mip_superposition_show(inference_array, pet_array, 0, uid, vmin = 0, vmax = 6, cmap_pet = 'Greys', cmap_mask = 'hsv', save = True, directory = mip_directory))
    sub.append(uid)
    filenames.append(sub)

In [ ]:
outpout_path_name = mip_diectory+'/'+'RELEVANCE_MIP'
create_pdf_mip(filenames, output_path_name)